# Exercice 1

### a) Convolution 1d

In [13]:
import numpy as np

In [14]:
input_data = np.array([1, 3, -2, 0, 2, -1, 3, 1, 2])
kernel = np.array([2, 1, -1])
bias = 2

conv_parameters = [(1,0),(2,0),(4,0),(1,1),(4,1)] # (stride, padding)

In [15]:
def conv1d(input_data, kernel, bias, stride=1, padding=0):
    
    input_padded = np.pad(input_data, (padding, padding), 'constant', constant_values=0)

    output_length = ((input_padded.size - kernel.size + 2 * padding) // stride) + 1
    
    # Avoid negative output length
    if output_length < 0:
        return np.array([])  
    
    output = []
    for i in range(0, len(input_padded) - len(kernel) + 1, stride):
       
        window = input_padded[i:i+len(kernel)]    
        if len(window) == len(kernel):
            output.append(np.sum(window * kernel) + bias)
    
    return np.array(output)

In [16]:
for stride, padding in conv_parameters:
    output = conv1d(input_data, kernel, bias, stride=stride, padding=padding)
    print(f"Stride: {stride}, Padding: {padding}\nOutput: {output}\n")

Stride: 1, Padding: 0
Output: [ 9  6 -4  5  2  2  7]

Stride: 2, Padding: 0
Output: [ 9 -4  2  7]

Stride: 4, Padding: 0
Output: [9 2]

Stride: 1, Padding: 1
Output: [ 0  9  6 -4  5  2  2  7  6]

Stride: 4, Padding: 1
Output: [0 5 6]


**Question :**
For which values of S and P we get an output of the same dimension as the input ?
Here's the general formula:
$$
\text{Output length} = \frac{\text{Input length} - \text{Filter size} + 2 \times \text{Padding}}{\text{Stride}} + 1
$$

And here's the formula with our values:
$$
\text{9} = \frac{\text{6} + 2 \times \text{Padding}}{\text{Stride}} + 1
$$

So if we have a basic stride of 1 we'll need a padding of 1 :
$$
\text{9} = \frac{\text{6} + 2 \times \text{1}}{\text{1}} + 1
$$

However, a lot of solution are possible for example if Stride = 2 and Padding = 5 :
$$
\text{9} = \frac{\text{6} + 2 \times \text{5}}{\text{2}} + 1
$$


### b) Convolution 2d

In [37]:
# Input array representing a 4x4 image with 3 channels
input_array = np.array([
    # Red channel
    [
        [2, 1, 0, 2],
        [2, 0, 1, 2],
        [0, 2, 1, 2],
        [0, 0, 1, 0]
    ],
    # Green channel
    [
        [0, 2, 2, 1],
        [0, 2, 2, 1],
        [2, 0, 2, 0],
        [1, 1, 0, 2]
    ],
    # Blue channel
    [
        [0, 2, 0, 0],
        [2, 1, 1, 1],
        [2, 0, 0, 2],
        [0, 2, 0, 2]
    ]
])

# Filters
filter1_array = np.array([
    [[1, 1], [0, 0]],  
    [[0, -1], [0, -1]],
    [[1, 0], [0, 1]]
])

filter2_array = np.array([
    [[0, 1], [1, 0]], 
    [[1, 0], [1, 0]],
    [[-1, 0], [0, 1]]
])

# Bias values
bias1 = 1
bias2 = -1

In [43]:
def conv2d(input_array, filter_array, bias, stride=1, padding=0):
  
    # Retrieve dimensions
    input_array_padded = np.pad(input_array, ((0, 0), (padding, padding), (padding, padding)), mode='constant', constant_values=0)
    num_channels, padded_height, padded_width = input_array_padded.shape
    _ ,filter_height, filter_width = filter_array.shape
    
    # Compute weight and width of the output
    output_height = ((padded_height - filter_height) // stride) + 1
    output_width = ((padded_width - filter_width) // stride) + 1
    
    output = np.zeros((output_height, output_width))
    
    # Run the convolution
    for channel in range(num_channels):
        for i in range(0, output_height):
            for j in range(0, output_width):
                # Calculate the start and end points for the current "slide"
                start_i = i * stride
                start_j = j * stride
                end_i = start_i + filter_height
                end_j = start_j + filter_width
                
                # Apply the filter
                output[i, j] += np.sum(input_array_padded[channel, start_i:end_i, start_j:end_j] * filter_array[channel])
                
    # Add of bias
    output += bias
    
    return output

In [44]:
output1 = conv2d(input_array, filter1_array, bias1,stride=1,padding=0)
output2 = conv2d(input_array, filter2_array, bias2,stride=1,padding=0)

print("Activation map 1 :\n", output1)
print("\nActivation map 2 :\n", output2)

Activation map 1 :
 [[ 1.  1.  2.]
 [ 3. -1.  6.]
 [ 6.  2.  4.]]

Activation map 2 :
 [[ 3.  2.  7.]
 [-1.  3.  7.]
 [ 4.  1.  6.]]


**Questions :**
a) How many activations map will we obtain ? 
- We'll obtain 2 activation map because we have 2 filters and each one reduce the outputs from 3  to 1 channel

Given two filters with dimensions (3 channels, 2 height, 2 width) and an input with dimensions (3 channels, 4 height, 4 width):
b) With S = 1 and P = 0, what will be the shape of the output ?
- With a stride (S) of 1 and no padding (P), the output dimensions are:
    - Output Height = $\frac{4 - 2 + 0}{1} + 1 = 3$
    - Output Width = $\frac{4 - 2 + 0}{1} + 1 = 3$
    - The shape of the output will be (2, 3, 3), as we have 2 activation maps.
    
c) With S = 2 and P = 0, what will be the shape of the output ?
- With a stride (S) of 2 and no padding (P), the output dimensions are calculated as:
    - Output Height = $\frac{4 - 2 + 0}{2} + 1 = 2$
    - Output Width = $\frac{4 - 2 + 0}{2} + 1 = 2$
    - The shape of the output will be (2, 2, 2), because we have 2 activation maps.
        
d) Give a filter size, padding value and stride value that will preserve the shape of the input.
- If we keep the same filter size and use a padding and a stride of the value of 1 we'll preserve the shape of the input.

e) Compute the values of the output with S = 1 and P = 0 using an appropriate iPython notebook.
- See code 2b)